In [1]:
# Imports
import numpy.typing as npt
import numpy as np
import pandas as df
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [2]:
# Linear Function for use in Curve Fit
def linearFunc(x: float, a: float) -> float:
    return a * x

# Define Dataset Class
class dataSet:
    partNum: int
    amps: npt.NDArray[np.float64] = np.array([0], dtype=float) # [A]
    meanV: npt.NDArray[np.float64] = np.array([0], dtype=float) # [mV]

    resist: float # [Ohm * m]
    cond: float # [S/m]

    # Units in [mm] and [mm^2]
    length: float
    side1: float
    side2: float
    area: float

    # Methods :
    def __init__(self, prt: npt.NDArray[np.float64]) -> None:
        self.partNum = int(prt[0][0][-1]) # Get Part Num
        prt = np.delete(prt, (0), axis=0) # Remove Header
        len = prt.shape[0]

        curr: int = 0
        count: int = 0
        for i in range(0,len):
            prt[i][0] = float(prt[i][0]) # Make everything floats
            if self.amps[curr] != prt[i][0] :
                if count > 0 :
                    self.meanV[curr] /= count
                self.amps = np.append(self.amps,prt[i][0])
                self.meanV = np.append(self.meanV,0)
                curr += 1
                count = 0
            self.meanV[curr] += abs(prt[i][1] - prt[i][2]) # Calculate the actual ΔV from Measurement and Baseline
            count += 1
        else: # Divide last meanV
            self.meanV[curr] /= count
        self.amps = np.delete(self.amps,0)
        self.meanV = np.delete(self.meanV,0)

    def setSize(self, length:float, side1: float, side2: float) -> None:
        self.length = length
        self.side1 = side1
        self.side2 = side2
        self.area = side1 * side2

    def calcConductivity(self) -> float:
        R: list[float] = curve_fit(linearFunc, self.amps, self.meanV * 1e-3)[0]
        

        self.resist = R[0] * (self.area * 1e-6) / (self.length * 1e-3)
        self.cond = 1 / self.resist
        return self.cond


# Read Data
part1 = dataSet(df.read_csv("part1.csv").to_numpy())
part2_1 = dataSet(df.read_csv("part2_1.csv").to_numpy())
part2_2 = dataSet(df.read_csv("part2_2.csv").to_numpy())
part3 = dataSet(df.read_csv("part3.csv").to_numpy())

p1_size1: float = np.mean([3.00, 3.05, 3.12, 3.10])
p1_size2: float = np.mean([2.74, 2.73, 2.77, 2.76])
p2_size1: float = np.mean([3.69, 3.70, 3.72, 3.69])
p2_size2: float = np.mean([3.01, 3.05, 3.04, 3.05])
p3_size1: float = np.mean([3.18, 3.09, 3.20, 3.10])
p3_size2: float = np.mean([2.24, 2.16, 2.13, 2.14])

part1.setSize(67.58, p1_size1, p1_size2)
part2_1.setSize(60.80, p2_size1, p2_size2)
part2_2.setSize(61.68, p2_size1, p2_size2)
part3.setSize(64.75, p3_size1, p3_size2)

cond: npt.NDArray[np.float64] = np.array([])

cond = np.append(cond, part1.calcConductivity())
cond = np.append(cond, part2_1.calcConductivity())
cond = np.append(cond, part2_2.calcConductivity())
cond = np.append(cond, part3.calcConductivity())
print(np.mean(cond))

7476688.272623353
